In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
import gc

In [3]:
df = pd.read_csv("/home/aban/somu/review-2/processed_data.csv")
df.drop(columns='Unnamed: 0',inplace=True)
df.columns = ['Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min',
       'Bwd Pkt Len Mean', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Bwd IAT Tot',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Pkts/s', 'Bwd Pkts/s',
       'Pkt Len Min', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt',
       'URG Flag Cnt', 'Down/Up Ratio', 'Fwd Seg Size Avg',
       'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Fwd Seg Size Min',
       'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Min','Label']
for column in df:
     if df[column].dtype == 'float64':
        df[column]=pd.to_numeric(df[column], downcast='float')
     if df[column].dtype == 'int64':
        df[column]=pd.to_numeric(df[column], downcast='integer')


In [4]:
train_X = df.drop(columns="Label",axis=1)
train_Y = df['Label']


In [5]:
del(df)
gc.collect()


0

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [7]:
params={'n_estimators' : [100,200,300],
       'max_depth' : [10,20,40]
       }

x_clf_4 = RandomForestClassifier(n_estimators=50,max_depth=10)
x_clf_4.fit(train_X,train_Y)


# random_clf_4 = RandomizedSearchCV(x_clf_4, param_distributions=params, scoring='f1', verbose=1, cv=2,n_jobs=-1)
# random_clf_4.fit(train_X,train_Y)


RandomForestClassifier(max_depth=10, n_estimators=50)

In [10]:
ypred = x_clf_4.predict(train_X)
f1_sc = f1_score(train_Y,ypred)
print(f1_sc)


0.9605316237509387


In [11]:
test_data = pd.read_csv("/home/aban/somu/test_prepro.csv")
test_data=test_data.drop(columns="Unnamed: 0",axis=1)


In [13]:
test_data.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Bwd Pkt Len Max,...,Down/Up Ratio,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min
0,443.0,6.0,1653.0,3.0,0.0,77.0,46.0,0.0,25.666667,0.0,...,0.0,25.666667,255.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
1,443.0,6.0,713.0,3.0,0.0,77.0,46.0,0.0,25.666667,0.0,...,0.0,25.666667,258.0,-1.0,20.0,0.0,0.0,0.0,0.0,0.0
2,443.0,6.0,20623981.0,8.0,7.0,740.0,388.0,0.0,92.500000,1460.0,...,0.0,92.500000,8192.0,7483.0,20.0,231049.0,0.0,231049.0,231049.0,20392473.0
3,53.0,17.0,1321.0,1.0,1.0,33.0,33.0,33.0,33.000000,87.0,...,1.0,33.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0
4,53.0,17.0,745.0,1.0,1.0,47.0,47.0,47.0,47.000000,113.0,...,1.0,47.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_pred_rf= x_clf_4.predict(test_data)

In [15]:
import pickle

In [16]:
pickle.dump(x_clf_4,open('./rafmodel.sav','wb'))

In [17]:
ypred_rf = pd.DataFrame(y_pred_rf)

In [19]:
ypred_rf = ypred_rf[0].astype(np.int8)

In [20]:
ypred_rf.head()

0    0
1    0
2    0
3    0
4    0
Name: 0, dtype: int8

In [21]:
ypred_rf.to_csv("./rf_pred_test.csv")